This notebook generates SMILES embeddings with MolR.
The code below was found here: https://github.com/hwwang55/MolR/blob/master/src/featurizer.py and slightly adapted.

# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Directory to the project folder
deep_learning_dir = '/content/gdrive/My Drive/BMI 707 Project' 

In [ ]:
! pip install dgl
! pip install pysmiles

  Using cached pbr-5.8.1-py2.py3-none-any.whl (113 kB)
  Created wheel for pysmiles: filename=pysmiles-1.0.1-py2.py3-none-any.whl size=22028 sha256=22787807f2bcf03aa2a4a6fcc6deb32a550c21baf6d2b287955d31cf83748594
  Stored in directory: /root/.cache/pip/wheels/e4/f0/ca/dae2e932684a6e26824d29cf5b6dadea7320e6fed036942972
Successfully built pysmiles


In [ ]:
!git clone https://github.com/hwwang55/MolR

Cloning into 'MolR'...
remote: Enumerating objects: 332, done.
remote: Total 332 (delta 0), reused 0 (delta 0), pack-reused 332
Receiving objects: 100% (332/332), 128.95 MiB | 28.70 MiB/s, done.
Resolving deltas: 100% (151/151), done.


In [ ]:
import os
os.chdir("MolR/src")

In [ ]:
import pickle
import pandas as pd
import dgl
import torch
import pysmiles
import numpy as np
from model import GNN
from dgl.dataloading import GraphDataLoader
from data_processing import networkx_to_dgl

# Generating embeddings from MolR

In [ ]:
class GraphDataset(dgl.data.DGLDataset):
    def __init__(self, path_to_model, smiles_list, gpu):
        self.path = path_to_model
        self.smiles_list = smiles_list
        self.gpu = gpu
        self.parsed = []
        self.graphs = []
        super().__init__(name='graph_dataset')

    def process(self):
        with open(self.path + '/feature_enc.pkl', 'rb') as f:
            feature_encoder = pickle.load(f)
        for i, smiles in enumerate(self.smiles_list):
            try:
                raw_graph = pysmiles.read_smiles(smiles, zero_order_bonds=False)
                dgl_graph = networkx_to_dgl(raw_graph, feature_encoder)
                self.graphs.append(dgl_graph)
                self.parsed.append(i)
            except:
                print('ERROR: No. %d smiles is not parsed successfully' % i)
        print('the number of smiles successfully parsed: %d' % len(self.parsed))
        print('the number of smiles failed to be parsed: %d' % (len(self.smiles_list) - len(self.parsed)))
        if torch.cuda.is_available() and self.gpu is not None:
            print(self.gpu)
            self.graphs = [graph.to('cuda:' + str(self.gpu)) for graph in self.graphs]

    def __getitem__(self, i):
        return self.graphs[i]

    def __len__(self):
        return len(self.graphs)


class MolEFeaturizer(object):
    def __init__(self, path_to_model, gpu=0):
        self.path_to_model = path_to_model
        self.gpu = gpu
        with open(path_to_model + '/hparams.pkl', 'rb') as f:
            hparams = pickle.load(f)
        self.mole = GNN(hparams['gnn'], hparams['layer'], hparams['feature_len'], hparams['dim'])
        self.dim = hparams['dim']
        if torch.cuda.is_available() and gpu is not None:
            self.mole.load_state_dict(torch.load(path_to_model + '/model.pt',map_location='cuda:0'))
            self.mole = self.mole.cuda(gpu)
        else:
            self.mole.load_state_dict(torch.load(path_to_model + '/model.pt', map_location=torch.device('cpu')))

    def transform(self, smiles_list, batch_size=None):
        data = GraphDataset(self.path_to_model, smiles_list, self.gpu)
        dataloader = GraphDataLoader(data, batch_size=batch_size if batch_size is not None else len(smiles_list))
        all_embeddings = np.zeros((len(smiles_list), self.dim), dtype=float)
        flags = np.zeros(len(smiles_list), dtype=bool)
        res = []
        with torch.no_grad():
            self.mole.eval()
            for graphs in dataloader:
                graph_embeddings = self.mole(graphs)
                res.append(graph_embeddings)
            res = torch.cat(res, dim=0).cpu().numpy()
        all_embeddings[data.parsed, :] = res
        flags[data.parsed] = True
        print('done\n')
        return all_embeddings, flags


def example_usage():
    model = MolEFeaturizer(path_to_model='../saved/gcn_1024')
    embeddings, flags = model.transform(['C', 'CC', 'CCC'])
    print(embeddings)
    print(flags)

def write_pickle(obj,fn):
    with open(fn,"wb") as f:
        pickle.dump(obj,f)

In [ ]:
# Example

if __name__ == '__main__':
    model = MolEFeaturizer(path_to_model='../saved/gcn_1024')
    smiles_embed, flags = model.transform(["CC(C)(C)NCC(O)C1=CC(O)=CC(O)=C1"])
    print(smiles_embed, str(len(smiles_embed[0])))

the number of smiles successfully parsed: 1
the number of smiles failed to be parsed: 0
done

[[ -4.83611679  -5.65992928  -2.12850332 ... -12.23947334  -4.20470333
    7.49038601]] 1024


In [ ]:
# Running with all unique SMILES in C
if __name__ == '__main__':
    df = pd.read_csv(deep_learning_dir + "/data_formatting/unique_matched_smiles.csv")
    smiles_list = df['smiles'].values
    model = MolEFeaturizer(path_to_model='../saved/gcn_1024')
    smiles_embed, flags = model.transform(smiles_list)
    print(flags)
    embed_dict = {smile: smile_embed for smile, smile_embed in zip(smiles_list,smiles_embed)}
    write_pickle(embed_dict,deep_learning_dir + "/embeddings/SMILES_embedding.pkl")

Atom "[C@@]" contains stereochemical information that will be discarded.
Atom "[C@@]" contains stereochemical information that will be discarded.
Atom "[C@@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@@H]" contains stereochemical information that will be discarded.
Atom "[C@]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
E/Z stereochemical information, which is specifie

ERROR: No. 0 smiles is not parsed successfully
ERROR: No. 12 smiles is not parsed successfully
ERROR: No. 17 smiles is not parsed successfully
ERROR: No. 18 smiles is not parsed successfully


Atom "[C@]" contains stereochemical information that will be discarded.
Atom "[C@]" contains stereochemical information that will be discarded.
Atom "[C@@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
E/Z stereochemical information, which is specified by "\", will be discarded
E/Z stereochemical information, which is specified by "\", will be discarded
Atom "[C@@]" contains stereochemical information that will be discarded.
Atom "[C@]" contains stereochemical information that will be discarded.
Atom "[C@@H]" contains stereochemical information that will be discarded.
Atom "[C@@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@@]" contains stereochemical information that will be discarded.
Atom "[C@@]" contains stereochemical inform

ERROR: No. 24 smiles is not parsed successfully
ERROR: No. 25 smiles is not parsed successfully
ERROR: No. 28 smiles is not parsed successfully
ERROR: No. 29 smiles is not parsed successfully
ERROR: No. 34 smiles is not parsed successfully
ERROR: No. 35 smiles is not parsed successfully


Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@@H]" contains stereochemical information that will be discarded.
Atom "[C@@H]" contains stereochemical information that will be discarded.
E/Z stereochemical information, which is specified by "\", will be discarded
E/Z stereochemical information, which is specified by "\", will be discarded
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@@H]" contains stereochemical information that will be discarded.
Atom "[C@@H]" contains stereochemical information that will be discarded.
Atom "[C@@]" contains stereochemical information that will be discarded.
Atom "[C@@]" contains stereochemical information that will be discarded.
Atom "[C@]" contains stereochemical information that will be discarded.
Atom "[C@]" contains stereochemical info

...

ERROR: No. 840 smiles is not parsed successfully
ERROR: No. 845 smiles is not parsed successfully
ERROR: No. 846 smiles is not parsed successfully
ERROR: No. 856 smiles is not parsed successfully


Atom "[C@@]" contains stereochemical information that will be discarded.
E/Z stereochemical information, which is specified by "\", will be discarded
Atom "[C@]" contains stereochemical information that will be discarded.
E/Z stereochemical information, which is specified by "\", will be discarded
Atom "[C@@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
E/Z stereochemical information, which is specified by "\", will be discarded
E/Z stereochemical information, which is specified by "\", will be discarded
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@@]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical

ERROR: No. 857 smiles is not parsed successfully
ERROR: No. 859 smiles is not parsed successfully
ERROR: No. 867 smiles is not parsed successfully


Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information 

ERROR: No. 871 smiles is not parsed successfully
the number of smiles successfully parsed: 708
the number of smiles failed to be parsed: 172
done

[False  True  True  True  True  True  True  True  True  True  True  True
 False  True  True  True  True False False  True  True  True  True  True
 False False  True  True False False  True  True  True  True False False
  True  True  True  True  True  True  True  True  True  True  True  True
  True False False  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True False  True  True  True  True  True  True  True  True  True False
 False  True  True  True  True  True  True  True  True  True False  True
 False  True  True False False  True False  True False  True  True  True
 False  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True False  True  True  

* SMILES successfully parsed: 708.
* SMILES failed to be parsed: 172.